<a href="https://colab.research.google.com/github/Piotr94/XGBoost_California_Housing/blob/master/California_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
import pandas as pd
from numpy import linspace, geomspace
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, accuracy_score
from xgboost import XGBRegressor, XGBClassifier

In [0]:
california_data = fetch_california_housing()
df = pd.DataFrame(california_data.data, columns=california_data.feature_names)
df['VALUE'] = pd.Series(california_data.target)

**Looking up into data**

In [22]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,VALUE
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [23]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,VALUE
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [24]:

if all(df.notnull()):
  print("There is no missing values")

There is no missing values


There is no missing values so I don't have to fill them

Splitting data into 3 sets: train, valid (for early stopping) and test.

In [36]:
y = df.VALUE
X = df.drop('VALUE', 1)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.85)
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, train_size=(0.7/0.85))
print(len(train_X), len(valid_X), len(test_X))

14448 3096 3096


**Regression**

In [37]:
# Testing xgboost model with default parameters and early stopping as a baseline
model = XGBRegressor(objective='reg:squarederror', n_estimators=2000)
fit_params={"early_stopping_rounds":50,
            "eval_metric": "mae",
            "eval_set": [[valid_X, valid_y]],
            "verbose": False}
model.fit(train_X, train_y, **fit_params, )
predicted = model.predict(test_X)
print(mean_absolute_error(test_y, predicted))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.30274429510664264


In [38]:
importances = list(model.feature_importances_)
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(prediction_features, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse=True)
for pair in feature_importances:
  print(pair[0], pair[1])

MedInc 0.489
AveOccup 0.133
Longitude 0.116
Latitude 0.104
HouseAge 0.078
AveRooms 0.039
Population 0.021
AveBedrms 0.02


In [40]:
# Searching best hyperparameters
learning_rate  = [0.003, 0.01, 0.03, 0.1, 0.2]
max_depth = [int(x) for x in linspace(3, 30, 10)]
colsample_bytree = linspace(0.5, 1, 6)
subsample = linspace(0.2, 1, 5)

params_grid = {'learning_rate': learning_rate,
               'max_depth': max_depth,
               'colsample_bytree': colsample_bytree,
               'subsample': subsample}

model_searching = RandomizedSearchCV(model, params_grid, 25, n_jobs=None, cv=3, scoring='neg_mean_absolute_error')
model_searching.fit(train_X, train_y, **fit_params)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=2000,
                                          n_jobs=1, nthread=None,
                                          objective='reg:squarederror',
                                          random_state=0, reg...
                   iid='deprecated', n_iter=25, n_jobs=None,
                   param_distributions={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                            

In [41]:
print(model_searching.best_params_)

{'subsample': 0.6000000000000001, 'max_depth': 6, 'learning_rate': 0.03, 'colsample_bytree': 0.8}


In [42]:
model = model_searching.best_estimator_
model.fit(train_X, train_y, **fit_params)

predicted = model.predict(test_X)
print("Mean absolute error for the test data: ", mean_absolute_error(test_y, predicted))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Mean absolute error for the test data:  0.2818368100208152
